## Week 2: Refrigeration

Welcome to week two of our python heat transfer tutorials. This week we're going to be looking at another one of your homework questions from last week; looking at refrigeration cycles.

![Alt text](ts.png)

1.10. A refrigerator uses refrigerant-134a as the working fluid and operates on an ideal vapor-compression refrigeration cycle between 0.14 and 0.8 MPa. If the mass flow rate of the refrigerant is 0.05 kg/s, determine:
(a) the rate of heat removal from the refrigerated space and the power input to the compressor, 
(b) the rate of heat rejection to the environment, and 
(c) the COP of the refrigerator.

The first thing that we're going to do is import numpy, a handy array based python class, kind of like matlab. If you're interested, you can read more about numpy [here](http://www.numpy.org/)

In [4]:
import numpy as np

Now we're going to use numpy to import the thermal properties table from the csv file you downloaded with this tutorial, so that we can manipulate it and make it do our bidding

In [5]:
fridgetab = np.genfromtxt('thermoproperties.csv', dtype=None, delimiter = ",")

This imports the data as an numpy array. Numpy arrays are useful in this particular instance because we can call the sections of the data that we are interested in. Note that in numpy arrays indexing starts from 0

In [6]:
fridgetab[0:3, :]

array([[b'R134a - TetraFlouroEthane Saturation Properties - Pressure Table',
        b'', b'', b'', b'', b'', b'', b'', b'', b''],
       [b'Pressure', b'Temp', b'volume (m^3/kg)', b'', b'enthalpy (kJ/kg)',
        b'', b'', b'entropy (kJ/kg.K)', b'', b''],
       [b'kPa', b'\xc2\xb0C', b'vf', b'vg', b'hf', b'hfg', b'hg', b'sf',
        b'sfg', b'sg']], 
      dtype='|S64')

So now we have imported the table as a numpy array, called 'fridgetab'

Now we can call the data necessary for our question. So We know that we need the data for 0.14 and 0.8 MPa, how do we call it? we need to figure out where it sits in our numpy array. We know that the data in this case starts from the fifth row at 60 Pa:

In [7]:
fridgetab[3,:]

array([b'60', b'-36.9', b'0.0007098', b'0.3112', b'3.9', b'223.9',
       b'227.8', b'0.0164', b'0.9481', b'0.9645'], 
      dtype='|S64')

So we want $h_{1-4}$ and $s_{1-2}$:

$h_1 = h_{g@0.14MPa}$

$s_1 = s_{g@0.14MPa}$

$h_2 = h_{g@0.8MPa}$

$s_2 = s_1$

$h_3 = h_{f@0.8MPa}$

$h_4 \cong h_3$ (why $\cong$ and not = ?)

So we need the data for 0.8 and 0.14 MPa respectively. From examining the table, we can work out that their indices will correspond to 

$i = \frac{P-0.08}{0.02} + 4$ for $P \leq 0.36  (MPa)$

$i = \frac{P-0.4}{0.1} + 21$ for $0.4 \leq P \leq 1$ and 

$i = \frac{P-1}{0.2} + 27$  for $1 \leq P$

So, for example for the data at $P = 140 KPa$, we could do:

In [8]:
fridgetab[round((0.14 - 0.08)/0.02)+4, :]

array([b'140', b'-18.8', b'0.0007383', b'0.1402', b'27.1', b'212.1',
       b'239.2', b'0.111', b'0.8337', b'0.9446'], 
      dtype='|S64')

We can check which collumn contains what data from the second and third row of the table:

In [9]:
fridgetab[1:3, :]

array([[b'Pressure', b'Temp', b'volume (m^3/kg)', b'', b'enthalpy (kJ/kg)',
        b'', b'', b'entropy (kJ/kg.K)', b'', b''],
       [b'kPa', b'\xc2\xb0C', b'vf', b'vg', b'hf', b'hfg', b'hg', b'sf',
        b'sfg', b'sg']], 
      dtype='|S64')

So from this we can gather that, for example, the first column is Pressure (from row 1) in Kpa (from row 2)

The command np.where can be used to find the index for the specific elements we are trying to call, which can then be defined as variables:

In [10]:
hg = np.where(fridgetab==b'hg')[1][0]
hf = np.where(fridgetab==b'hf')[1][0]
sg = np.where(fridgetab==b'sg')[1][0]
print(sg)

9


Here we dont need to worry about the b'...' formatting of the input for now, it just allows python to match our search with the required location.

Let's define a function that calls the data for a given Pressure, we can do this using if loops to call the appropriate algorithm for a given pressure. You should remember if loops from matlab. Below is an example of if loop syntax in python (Remember, indentation matters!) 

In [11]:
a = 'frog'
b = 'frog'
if a == b:
    print('ribbit')

ribbit


Hint: We can also use boolean operators other than "==" in if loops. So here, for example you could try things like:

In [12]:
Pa = 120

if Pa <= 140:
    print("sheep")

elif Pa >= 140:
    print("cats")

sheep


So extending on our example above with $140 KPa$:

In [33]:
x = 1.8

if x < 0.36: # Here we are introducing the thresholds defined above
    c1 = 0.08 # These are the variables from above where the thresholds were defined
    c2 = 0.02
    c3 = 4()
elif 0.36 < x and x < 1:
    c1 = 0.4
    c2 = 0.1
    c3 = 19 
elif x > 1:
    c1 = 1
    c2 = 0.2
    c3 = 25

hg = np.where(fridgetab==b'hg')[1][0] # defining the locations for the values we need
hf = np.where(fridgetab==b'hf')[1][0]
sg = np.where(fridgetab==b'sg')[1][0]

i = round((x-c1)/c2 + c3) # calculating the index based on the above variables
print(float(fridgetab[i, hf])) # float() changes the output into an integer, which can then be used for calculations later

144.1


See if you can change the code above into a function, with input variables of Pressure and desired output:

In [14]:
def getindex(thresholds, funk, value, ntier=3):
    assert (len(thresholds) == ntier),"wrong number of thresholds"  # this line and the line below are just for catching errors in your input
    assert (ntier == len(funk)),"wrong number of functions"
    i = 0
    try:
        while value > thresholds[i]:
            i = i+1
    except IndexError:
        raise ValueError('your value\'s off the charts!')
    else:
        return round(funk[i](value)) + 4

def lookuptab(table, index, variable):
    return float(table[index][np.where(table==variable.encode('utf-8'))[1][0]])

lookuptab(fridgetab, getindex([0.36, 1, 3], 
                              [lambda x: (x-0.08)/0.02, 
                               lambda x: (x-0.4)/0.1+15,
                               lambda x: (x-1)/0.2 + 21], 1.2), 'hg')




273.9

Coming back to the question at hand

(a) the rate of heat removal from the refrigerated space and the power input to the compressor

we recall the definition for the rate of heat removal:

$$\dot{Q}_L = \dot{m}(h_1 - h_4)$$

Solving should now be trivial with our new function

During the week, try and rewrite the code above using a while loop instead of if loops. You can read more about while loops here